Extract cell type embeddings from layer prior to classification heads.

Prompting with all genes and no metadata.

In [1]:
import os
import math
import torch
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from braceexpand import braceexpand
from tqdm import tqdm

# for flex attention
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# DEVICE = torch.device('cuda:1')
DEVICE = torch.device('cuda:0')
sc.set_figure_params(figsize=(4, 4))

from cellarium.ml.utilities.inference.cellarium_gpt_inference import \
    CellariumGPTInferenceContext, \
    GeneNetworkAnalysisBase

2025-02-25 14:14:20.497059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-25 14:14:21.920141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ROOT_PATH = "/work/hdd/bbjr/mallina1/data/mb-ml-dev-vm"
TRAIN_ROOT_PATH = "/work/hdd/bbjr/mallina1/data/human_cellariumgpt_v2/extract_files"
CHECKPOINT_PATH = "/work/hdd/bbjr/mallina1/cellarium/models/latest/epoch=5-step=452000.ckpt"

REF_ADATA_PATH = os.path.join(ROOT_PATH, "data", "extract_0.h5ad")
GENE_INFO_PATH = os.path.join(ROOT_PATH, "gene_info", "gene_info.tsv")

ctx = CellariumGPTInferenceContext(
    cellarium_gpt_ckpt_path=CHECKPOINT_PATH,
    ref_adata_path=REF_ADATA_PATH,
    gene_info_tsv_path=GENE_INFO_PATH,
    device=DEVICE,
    attention_backend="mem_efficient"
)

Get the names of the 100 most expressed cell types in the first train chunk and filter down on these. To be updated later with a custom list.

In [ ]:
train_fnames = list(braceexpand('extract_{0..10}.h5ad'))

tr_adata = sc.read_h5ad(os.path.join(TRAIN_ROOT_PATH, train_fnames[0]))
cell_type_filter_list = list(tr_adata.obs['cell_type'].value_counts().to_dict().keys())[:11]
cell_type_filter_list.remove('unknown')

print(cell_type_filter_list)

cell_type_filter_list = set(cell_type_filter_list)

['neuron', 'L2/3-6 intratelencephalic projecting glutamatergic neuron', 'glutamatergic neuron', 'oligodendrocyte', 'CD4-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell', 'B cell', 'classical monocyte', 'natural killer cell', 'fibroblast']


: 

In [4]:
metadata_prompt_dict = {
    "cell_type": False,
    "tissue": False,
    "disease": False,
    "sex": False,
    "development_stage": False
}

train_fnames = train_fnames[:1]

cell_type_embeddings = []
disease_embeddings = []

susp_type_labels = []
assay_labels = []
cell_type_labels = []
disease_labels = []

batch_size = 3
for train_fname in train_fnames:
    tr_adata = sc.read_h5ad(os.path.join(TRAIN_ROOT_PATH, train_fname))

    # only preserve rows where the cell type is in the list we want
    tr_adata_filtered = tr_adata[tr_adata.obs.cell_type.isin(cell_type_filter_list)]

    for obs_idx in tqdm(range(0, tr_adata_filtered.obs.shape[0], batch_size)):
        try:
            tokens_dict, context_indices = ctx.generate_tokens_from_adata(tr_adata_filtered, obs_index=[obs_idx, obs_idx+1, obs_idx+2], query_var_names=[],
                                                                        metadata_prompt_masks_dict=metadata_prompt_dict)
        except:
            # incomplete batch, stop iterating
            break

        query_cell_type_idx = context_indices['query_cell_type']
        query_disease_idx = context_indices['query_disease']

        with torch.inference_mode():
            hidden_states_ncd, _ = ctx.get_embeddings_from_tokens(tokens_dict, context_indices)
            hidden_states_ncd = hidden_states_ncd.cpu()

        cell_type_embeddings.append(hidden_states_ncd[:, query_cell_type_idx, :])
        disease_embeddings.append(hidden_states_ncd[:, query_disease_idx, :])

        cell_type_labels.append(tr_adata_filtered.obs.iloc[obs_idx].cell_type)
        disease_labels.append(tr_adata_filtered.obs.iloc[obs_idx].disease)
        susp_type_labels.append(tr_adata_filtered.obs.iloc[obs_idx].suspension_type)
        assay_labels.append(tr_adata_filtered.obs.iloc[obs_idx].assay)

        # if len(cell_type_embeddings) >= math.ceil(1000/batch_size):
        #     break

cell_type_embeddings = torch.cat(cell_type_embeddings, dim=0).squeeze()
disease_embeddings = torch.cat(disease_embeddings, dim=0).squeeze()


 41%|████████████████████████████████████████▌                                                         | 436/1053 [2:15:25<3:11:15, 18.60s/it]

: 

: 

In [1]:
cell_type_embeddings.shape
disease_embeddings.shape

NameError: name 'cell_type_embeddings' is not defined